In [1]:
import json
from tqdm import tqdm

In [2]:
big = "memoryalpha"
small = "stexpanded"
top_k = 10

gold_pairs_file = "./data/gold_pairs/" + big + "-" + small + "_formatted.txt"
matched_communities_file = "./results/_matched_communities_leiden/top_10_pairs/" + big + "-" + small + "_top_10_pairs.json"

exact_match_file = "./data/exact_match/" + big + "-" + small + "-formatted.json"

communities_big_file = "./results/communities_leiden/" + big + "/final.txt"
communities_small_file = "./results/communities_leiden/" + small + "/final.txt"

small_communities = []
big_communities = []
gold_pairs = []
matched_communities = []

In [3]:
with (open(communities_big_file) as cbf, open(communities_small_file) as csf,
      open(gold_pairs_file) as gpf, open(matched_communities_file) as mcf):
    
    for line in csf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        small_communities.append(numbers_set)

    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        big_communities.append(numbers_set)
    
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

    matched_communities_load = json.load(mcf)
    matched_communities = [[int(key), [int(values[i][0]) for i in range(0, top_k)]] for key, values in matched_communities_load.items()]

In [4]:
matched_communities

[[0, [0, 646, 1157, 451, 305, 1113, 900, 469, 351, 1112]],
 [1, [667, 648, 1113, 1111, 656, 661, 666, 1017, 647, 402]],
 [2, [336, 338, 478, 441, 389, 435, 431, 479, 340, 439]],
 [3, [5, 609, 370, 648, 608, 616, 401, 611, 667, 606]],
 [4, [536, 761, 507, 436, 1139, 760, 432, 758, 104, 856]],
 [5, [817, 759, 64, 356, 761, 423, 536, 347, 104, 452]],
 [6, [356, 536, 507, 423, 758, 760, 473, 81, 1005, 556]],
 [7, [436, 428, 425, 278, 554, 473, 760, 784, 536, 507]],
 [8, [441, 278, 536, 507, 1139, 856, 434, 576, 546, 849]],
 [9, [784, 356, 574, 436, 774, 856, 760, 507, 775, 104]],
 [10, [536, 507, 473, 576, 937, 760, 1139, 758, 81, 556]],
 [11, [507, 758, 937, 536, 556, 760, 576, 446, 1139, 546]],
 [12, [761, 278, 536, 436, 759, 1139, 758, 757, 762, 356]],
 [13, [815, 477, 19, 960, 370, 349, 348, 347, 698, 351]],
 [14, [379, 1, 806, 1088, 358, 88, 807, 809, 14, 808]],
 [15, [1092, 1093, 1, 807, 839, 823, 46, 806, 278, 829]],
 [16, [83, 846, 1092, 698, 829, 807, 1, 280, 12, 278]],
 [17, [536

In [5]:
with open(exact_match_file) as file:
    exact_match = json.load(file)

gold_not_exact = list()
for p in gold_pairs:
    if [p[0], p[1]] not in exact_match:
        gold_not_exact.append([p[0], p[1]])

In [6]:
found = 0
# all_pairs = len(gold_pairs)
all_pairs = len(gold_not_exact)

# for gold_pair in tqdm(gold_pairs):
for gold_pair in tqdm(gold_not_exact):
    for matched_community in matched_communities:
        community_1 = big_communities[matched_community[0]]
        community_2 = set()
        for i in range(0, top_k):
            community_2.update(small_communities[matched_community[1][i]])
        if gold_pair[0] in community_1 and gold_pair[1] in community_2:
            found += 1
            # print(matched_community[0], matched_community[1], gold_pair)
            break
            
print("Gold pairs in matched communities top " + str(top_k) + ": " + str(found / all_pairs * 100) + "%")

100%|██████████| 148/148 [00:04<00:00, 31.31it/s]

Gold pairs in matched communities top 10: 33.78378378378378%


In [7]:
found_big = 0
found_small = 0
for gold_pair in gold_not_exact:
    for big_community in big_communities:
        if gold_pair[0] in big_community:
            found_big += 1
            break
    for small_community in small_communities:
        if gold_pair[1] in small_community:
            found_small += 1
            break

print("Big: " + str(found_big / all_pairs * 100) + "%")
print("Small: " + str(found_small / all_pairs * 100) + "%")

Big: 100.0%
Small: 100.0%
